# Contents
1. Download kaggle datasets
2. Scrape following from MAL websites:
    - Synopsis
    - Background
    - Voice actors (TBC)
    - Image links

Data preprocessing intended for next ipynb

## 1. Download kaggle datasets

In [2]:
import kaggle
import pandas as pd


ds_path = "../data/01_raw"

kaggle.api.authenticate()

In [3]:
ds1_kagglename = "CooperUnion/anime-recommendations-database"
ds2_kagglename = "crazygump/myanimelist-scrappind-a-decade-of-anime"

kaggle.api.dataset_download_files(ds1_kagglename, path=ds_path, unzip=True)
kaggle.api.dataset_download_files(ds2_kagglename, path=ds_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database
Dataset URL: https://www.kaggle.com/datasets/crazygump/myanimelist-scrappind-a-decade-of-anime


## 2. Scrape data from MAL Websites

In [30]:
import pandas as pd 

# Gathering unique anime_id values from ratings dataset:
ratings_path = "../data/01_raw/rating.csv"
df_ratings = pd.read_csv(ratings_path)

# df_unique_anime = df_ratings["anime_id"].unique()
df_unique_anime = df_ratings.drop_duplicates(subset=['anime_id'])
print(len(df_unique_anime))

11200


In [43]:
partition_size = int(len(df_unique_anime)/4)
df_unique_anime_damien = df_unique_anime[:partition_size]
df_unique_anime_leroy = df_unique_anime[partition_size:2*partition_size]
df_unique_anime_rosamund = df_unique_anime[2*partition_size:3*partition_size]
df_unique_anime_kenneth = df_unique_anime[3*partition_size:4*partition_size]

print(df_unique_anime_damien.info())
print(df_unique_anime_leroy.info())
print(df_unique_anime_rosamund.info())
print(df_unique_anime_kenneth.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2800 entries, 0 to 15666
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   2800 non-null   int64
 1   anime_id  2800 non-null   int64
 2   rating    2800 non-null   int64
dtypes: int64(3)
memory usage: 87.5 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 2800 entries, 15667 to 146310
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   2800 non-null   int64
 1   anime_id  2800 non-null   int64
 2   rating    2800 non-null   int64
dtypes: int64(3)
memory usage: 87.5 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 2800 entries, 146312 to 1158091
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   2800 non-null   int64
 1   anime_id  2800 non-null   int64
 2   rating    2800 non-null   int64
dtypes: int64(3)
memory usage: 87.5 KB


In [ ]:
# Export unique anime IDs for separate scrapy scraping
df_unique_anime_damien.to_csv("../data/01_raw/rating_damien.csv")
df_unique_anime_leroy.to_csv("../data/01_raw/rating_leroy.csv")
df_unique_anime_rosamund.to_csv("../data/01_raw/rating_rosamund.csv")
df_unique_anime_kenneth.to_csv("../data/01_raw/rating_kenneth.csv")